In [ ]:
import pandas as pd
import sklearn
import numpy as np


In [ ]:
data = pd.read_csv("out.csv").dropna()

# Modify the 'winner' column based on conditions
data['winner'] = data.apply(lambda row: 1 if row['winner'] == row['fighter_1'] else 0, axis=1)


# Drop the specified columns
columns_to_drop = ['fight_id', 'fighter_1_id', 'fighter_1', 'fighter_2', 'fighter_2_id','decision_method','fight_duration_lastrnd','time_format','date', 'location','height_x','name_x','weight_x','height_y','name_y','weight_y' ]
data_cleaned = data.drop(columns=columns_to_drop)

# Display the first few rows of the cleaned data
data_cleaned.head()

data_cleaned.to_csv("cleanedData2.csv", index = False)

In [ ]:
#split up the data

winners = data_cleaned.iloc[:,0]
features = data_cleaned.iloc[:,1]


features = features.sample(frac =1 )



In [ ]:
df = pd.read_csv("encoded_cleanedDataNEW.csv") # original dataset
##print(f"Initial shape: {df.shape}")

'''keep = np.array([4,2,1,6,10,11,12,37,38,63,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,104,105,130,132,133,134,135,136,137,138,139,140,141,142,143,144,145]) - 1
df = df.iloc[:,:] # interesting fields

df["date"] = pd.to_datetime(df["date"]) # date as datetime


df = df.replace(r'^\s*$', np.nan, regex=True) # replace empty string with nan
df = df.fillna(np.nan) # Fill empty and NaNs values with NaN

df = df.round(3) # format data

df.rename(columns={ # some renaming
    "B_win_by_KO/TKO": "B_win_by_KO_TKO",
    "R_win_by_KO/TKO": "R_win_by_KO_TKO"},inplace=True)
##print(f"shape: {df.shape}")

# considering matches with a winner as this will be a binary classification
drawRate = round(len(df["Winner"][df["Winner"]== "Draw"])/len(df.index)*100,3)
print(f"Draw rate is: {drawRate}%, converting to nan")
df["Winner"] = df["Winner"].replace("Draw", np.nan) # convert to nan to be dropped later
##df.dropna()'''



df

In [ ]:
##shuffling the data 
df_shuffled = df.sample(frac = 1)
df_shuffled.head(n=5)

In [ ]:
##split up the data 
df_shuffled.columns

'''x = df_shuffled.drop([], axis = 1).values
y = df_shuffled["Winner"].values'''
y = df_shuffled.iloc[:,0]
x = df_shuffled.iloc[:,1]

In [ ]:
import numpy as np

class CustomMinMaxScaler:
    def __init__(self):
        self.min_ = None
        self.max_ = None

    def fit(self, data):
        # Compute min and max values for each feature
        self.min_ = np.min(data, axis=0)
        self.max_ = np.max(data, axis=0)

    def transform(self, data):
        # Scale data to the range [0, 1]
        return (data - self.min_) / (self.max_ - self.min_)

    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
xTrain, xTest, yTrain, yTest = train_test_split(x,y,test_size = .5)

In [ ]:
'''winners = y = df_shuffled["Winner"].values'''

scaler = CustomMinMaxScaler()

xTrainScaled = scaler.fit_transform(xTrain)
xTestScaled = scaler.fit_transform(xTest)



In [ ]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression(penalty = 'l1', tol = 1, solver = 'liblinear',
                            multi_class = "auto",
                            fit_intercept = False, 
                            max_iter =3)
model1.fit(xTrainScaled, yTrain)
joblib.dump(model1, 'model1.joblib')


In [ ]:
testing_model = joblib.load("model1.joblib")
prediction = testing_model.predict(xTestScaled)
print(prediction)
for i in prediction:
    print(i)

In [ ]:
print("Accuracy score for Logistic Regression Model: %s" % (sklearn.metrics.accuracy_score(yTest,prediction)))
print("Precision score for logistic Regression Model: %s" % (sklearn.metrics.precision_score(yTest,prediction)))
print("Recall score for Logistic Regression Model: %s" % (sklearn.metrics.recall_score(yTest,prediction)))
print("F1 score for Logistic Regression Model: %s" % (sklearn.metrics.f1_score(yTest,prediction)))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model2 = KNeighborsClassifier(n_neighbors = 4, p =3)
model2.fit(xTrainScaled,yTrain)
joblib.dump(model2, 'model2.joblib')

In [ ]:
knnModel = joblib.load("model2.joblib")
knnPrediction = knnModel.predict(xTestScaled)
for i in range(len(knnPrediction)): 
    print("Model 2 prediction based on data B for item: %d : %s" % (i + 1, knnPrediction[i]))

In [ ]:
knnModel_c = joblib.load("model2.joblib")
knnPrediction_c = knnModel_c.predict(xTestScaled)

C_answers = yTest
TP, FP = 0,0
#for i in range(len(knnPrediction_c)): 
    #print("Model 2 prediction based on data C: %s" % (knnPrediction_c[i]))
print("Accuracy Score for KNN Model: %s" %(sklearn.metrics.accuracy_score(C_answers,knnPrediction_c)))
print("Precision score for KNN Model: %s" %(sklearn.metrics.precision_score(C_answers,knnPrediction_c)))
print("Recall Score for KNN Model: %s" % (sklearn.metrics.recall_score(C_answers,knnPrediction_c)))
print("F1 Score for KNN Model: %s" % (sklearn.metrics.f1_score(C_answers,knnPrediction_c)))
FDR_knn = 1 - sklearn.metrics.precision_score(C_answers, knnPrediction_c)
print("False Detection Rate for KNN Model: %s" %(FDR_knn))
print("Matthews Coeffiecen for KNN Model: %s" % (sklearn.metrics.matthews_corrcoef(C_answers,knnPrediction_c)))

print(f'=====================================================================================================')

In [ ]:
from sklearn.neural_network import MLPClassifier
model3 = MLPClassifier(solver = "lbfgs", alpha=.1, hidden_layer_sizes=(1,5))
model3.fit(xTrainScaled,yTrain)
joblib.dump(model3, 'model3.joblib')

In [ ]:
mlpModel = joblib.load("model3.joblib")
mlpPrediction = mlpModel.predict(xTestScaled)
for i in range(len(mlpPrediction)): 
    print("Model 4 prediction for item: %d : %s" % (i + 1, mlpPrediction[i]))

In [ ]:
#rand forest 
from sklearn.ensemble import RandomForestClassifier
model4 = RandomForestClassifier(max_depth = 2)
model4.fit(xTrainScaled,yTrain)
joblib.dump(model4,'model4.joblib')


In [ ]:
forestModel = joblib.load("model4.joblib")
forestPrediction = forestModel.predict(xTestScaled)
for i in range(len(forestPrediction)): 
    print("Model 6 prediction for item: %d : %s" % (i + 1, forestPrediction[i]))



In [ ]:
forestModel_c = joblib.load("model4.joblib")
forestPrediction_c= forestModel_c.predict(xTestScaled)
C_answers = yTest
TP, FP = 0,0
for i in forestPrediction_c: 
    #print("Model 6 prediction for item: %s" % (forestPrediction_c[i]))
    if forestPrediction_c[i] == C_answers[i]: 
        TP += 1
        
    elif forestPrediction_c[i] == 1 and C_answers[i]==0 or forestPrediction_c[i]==0 and C_answers[i]==1:
        FP += 1
print("Accuracy Score for Forest Model: %s" %(sklearn.metrics.accuracy_score(C_answers,forestPrediction_c)))
print("Precision Score for Forest Model: %s" %(sklearn.metrics.precision_score(C_answers,forestPrediction_c)))
print("Recall Score for Forest Model: %s" % (sklearn.metrics.recall_score(C_answers,forestPrediction_c)))
print("F1 Score for Forest Model: %s" %(sklearn.metrics.f1_score(C_answers,forestPrediction_c)))
FDR_forest = 1 - sklearn.metrics.precision_score(C_answers,forestPrediction_c)
print("False Detection Rate for Forest Model: %s" %(FDR_forest))
print("Matthews Coefficent for Forest Model: %s" % (sklearn.metrics.matthews_corrcoef(C_answers,forestPrediction_c)))
print(f'=====================================================================================================')

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import joblib

# Define the model
knn = KNeighborsClassifier()

# Set up the parameter grid
param_grid = {
    'n_neighbors': range(1, 21),
    'p': [1, 2, 3, 4, 5]  # You may add other values if you want to try custom distance metrics
}

# Set up Grid Search with cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(xTrainScaled, yTrain)

# Get the best model
best_model = grid_search.best_estimator_

# Save the optimized model
joblib.dump(best_model, 'optimized_model2.joblib')

# Print the best parameters and accuracy
print("Best parameters:", grid_search.best_params_)
print("Best cross-validated accuracy:", grid_search.best_score_)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

# Train the Linear Regression model
linear_model = LinearRegression()
linear_model.fit(xTrainScaled, yTrain)

# Predict on the training or test set
y_pred_continuous = linear_model.predict(xTrainScaled)

# Apply threshold rounding to get binary predictions
y_pred_binary = np.where(y_pred_continuous >= 0.5, 1, 0)

# Evaluate accuracy
accuracy = accuracy_score(yTrain, y_pred_binary)
print("Accuracy (rounded to binary):", accuracy)

# Save the model if desired
joblib.dump(linear_model, 'binary_linear_model.joblib')


In [28]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Define the parameters for GradientBoostingClassifier
n_estimators = 100       # Number of boosting stages (trees)
learning_rate = 0.1      # Step size at each iteration
max_depth = 3            # Maximum depth of the individual trees
subsample = 1.0          # Proportion of samples used to fit each tree
min_samples_split = 2    # Minimum number of samples required to split an internal node
min_samples_leaf = 1     # Minimum number of samples required to be at a leaf node
max_features = None      # Number of features to consider for the best split

# Assuming your data is preprocessed into X (features) and y (labels)

# Convert Pandas Series to NumPy arrays, if necessary
xTrain = xTrain.to_numpy() if isinstance(xTrain, pd.Series) else xTrain
yTrain = yTrain.to_numpy() if isinstance(yTrain, pd.Series) else yTrain

# Debugging: Check the shape of the input data
print(f"xTrain shape: {xTrain.shape}, yTrain shape: {yTrain.shape}")

# Ensure that X is a 2D array (n_samples, n_features) and y is a 1D array (n_samples,)
if len(xTrain.shape) == 1:  # If X has only one feature (1D array)
    xTrain = xTrain.reshape(-1, 1)  # Reshape it to be 2D (n_samples, 1)

if len(yTrain.shape) > 1:  # If y has more than one column
    yTrain = yTrain.reshape(-1, 1)  # Flatten it to be a 1D array

# Double-check the shape again after reshaping
print(f"xTrain shape after reshaping: {xTrain.shape}, yTrain shape after reshaping: {yTrain.shape}")

# Define the Gradient Boosting model with the specified parameters
model = GradientBoostingClassifier(
    n_estimators=n_estimators,
    learning_rate=learning_rate,
    max_depth=max_depth,
    subsample=subsample,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf,
    max_features=max_features
)

# Train the model on the training data
model.fit(xTrain, yTrain)

# Convert xTest to numpy if it is a pandas Series, then reshape if it's 1D
if isinstance(xTest, pd.Series):
    xTest = xTest.to_numpy()

# Reshape xTest if it's 1D
if len(xTest.shape) == 1:
    xTest = xTest.reshape(-1, 1)

# Make predictions using the trained model
y_pred = model.predict(xTest)

# Calculate the accuracy of the model
accuracy = accuracy_score(yTest, y_pred)

# Print the final results
print(f"Test Set Accuracy: {accuracy:.4f}")


xTrain shape: (1913, 1), yTrain shape: (1913,)
xTrain shape after reshaping: (1913, 1), yTrain shape after reshaping: (1913,)
Test Set Accuracy: 0.6050
